<a href="https://colab.research.google.com/github/gabifc/machine_learning_dell/blob/main/tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Tensor Flow  
Oferece uma alternativa mais complexa e robusta que permite a construção de redes com mais camadas e mais opções.

Abaixo o passo a passa para construir e avaliar um modelo.



In [2]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [3]:
# carregando conjunto de dados na variável
fashion_mnist = keras.datasets.fashion_mnist
# separar o conjunto de dados em treino e teste e carregar na variavel 
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

4423680/4422102 [==============================] - 0s 0us/step


In [4]:
train_images.shape
# Tensor 3d com 60 mil elementos e cada elemento é uma matriz de 28 por 28.

(60000, 28, 28)

In [5]:
len(train_labels) # para verificar o tamanho do conjunto dos rotulos de treino.

60000

In [6]:
np.unique(train_labels) 
# tenho todos os labels que aparecem neste conjunto. De 0 a 9 tenho 10 categorias diferentes. 
# Os 60 mil rotulos estão distribuidas nestas 10 classes. 

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=uint8)

In [7]:
test_images.shape # verificando o tamanho do teste. 10 mil imagens de 28 por 28.

(10000, 28, 28)

In [8]:
len(test_labels) # quantidade de rotulos de teste. Tem que bater a quantidade de elementos com a quantidade de rotulos.

10000

In [9]:
test_labels 
# o rotulo da primeira imagem de teste é 9, da segunda é 2...

array([9, 2, 1, ..., 8, 1, 5], dtype=uint8)

Para finalizar essa preparação de dados nós podemos verificar que as nbossas imagens de treino e teste s compostas de matriz de dimensoes de 28 po 28 . Cada um desses elementos da matriz é exatamente um pixel com valores entre 0 e 255.  
Onde 0 é a ausencia de cor no pixel e 255 é a intensidade maxima do pixel. Em alguns modelos, tais como a regressão linear e as próprias redes neurais se beneficiam de atributos escalados. Esses modelos utilizam uma técnica de gradiente para otimização ou seja de derivada. Portanto quando estes valores estão numa escala pequena como entre 0 e 1 esse modelos tem melhor perfomance. Como todos os pixels estão com valores entre 0 e 255 nós podemos dividir as nossas matrizes por 255 e dessa maneira o pixel com maior intensidade que é o de 255 será igual a 1 o piel de menor intensidade que é 0 continuará sendo 0 e todos os valores intermediários estará na escla entre 0 e 1.  
Estamos utilizando um MinMaxScaler chamado explicitamente.

In [10]:
train_images = train_images / 255.0

test_images = test_images / 255.0

Dividido por 255.0 para forçar o resultado a ser um ponto flutuante e todos os valores tanto das imagens de traino quanto as de teste estarão entre 0 e 1.  
Com isso teremos preparado o conjunto de dados para utilizar a rede neural. Esta preparação do conjunto de dados é tão importante quanto a implementação do modelo uma vez que dados que não estejam corretamente alinhados ou preparados podem gerar modelos ruins ou modelos cuja avaliação não é confiável.Então temos um modelo que é capaz de ser executado de forma confiável já que temos separado treino e teste para avaliação e estamos normalizando o nosso conjunto de dados.

#Construindo o modelo de Rede Neural com TensorFlow  
Modelo Sequencial, as camadas são em sequencia e o resultado de uma vai para proxima.  
Flatten - a matriz que tem tamanho 28 por 28 é traduzido 28 vezes 28. Pq esta rede não é capaz de trabalhar com matrizes. Então transforma a matriz em vetor concatenando as linhas dessa matriz. Só passar como parametro o tamanho do vetor de entrada.  
Dense - camadas densas, comuns de redes neurais que liga todos os neuronios da camada anterior aos neuronios da camada atual. Utilizo uma função de ativação para cada uma dessas camadas.  
Relu - função que transforma qualquer número negativo em 0 e se o nr for positivo mantém o nr, ou seja, elimina nrs negativos.  
Softmax - função de classificação, define as classes que permite a avaliação. 
Na última camada temos 10 neuronios e tbm 10 classes. A última camada da rede neural deve bater com o nr de classes 1 para 1 deve ser sempre assim.  
model.summary- quantos pesos temos na rede neural e a estrutura dela.

In [11]:
model = keras.Sequential([
    keras.layers.Flatten(input_shape=(28, 28)),
    keras.layers.Dense(128, activation=tf.nn.relu),
    keras.layers.Dense(10, activation=tf.nn.softmax)
])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 784)               0         
_________________________________________________________________
dense (Dense)                (None, 128)               100480    
_________________________________________________________________
dense_1 (Dense)              (None, 10)                1290      
Total params: 101,770
Trainable params: 101,770
Non-trainable params: 0
_________________________________________________________________


Modelos sequencial  

**Camada Flatten** - 784 que é 28 vezes 28 e Param # é 0 pq não temos nenhum parametro de peso. Estou apenas convertendo uma matriz em um vetor.  

**Camada Dense** - saída 128 exatamente igual ao número de neuronios nesta camada. Param # é 100480 exatamente 784 elementos de entrada dessa camada vezes os 128 neuronios mais 128. Lembrando que para cada elemento de entrada temos um peso(parametro) para cada neuronio, por isso a multiplicação de 784x128 e cada neuronio possui um fator de viés, ou *bias* (lê-se baias?). Este bias será somado pq é um peso e para cada um dos neuronios nós temos um *baias*, logo 784x128+128=100480 parametros.  

**Camada dense_1** - composta de 10 neuronios, a saida dessa camada será igual a 10 sendo 1 neuronio para cada classe e teremos 1290 parametros nessa classe 128x10=1280.  
Somo aos 100480 da camada anterior e no total teremos 101770 parametros que serão treináveis no passo de treinamento da nossa rede neural.


In [12]:
model.compile(optimizer='adam', 
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

Compilando o modelo utilizando os parametros:  

**optimizer='adam'** - é o otimizador para os pesos, é a tecnica utilizada para a construção dos pesos para o treinamento.  

**loss** - metrica de erro, entropia cruzada categórica esparsa. Tecnica que permite trabalhar com resultados que possui muitos valores próximos a 0 e alguns valores prox a 1. Que contempla os valores de saída destes neuronios.  

**metrics=['accuracy']** - porcentagem de elementos classificados corretamente sobre o total. Assim compila o modelo e já posso treinar.

In [13]:
# Treinando o modelo com 5 épocas.
# epochs=5 é 5 iterações dos nossos conjuntos de dados para construir um bom modelo. 
# Pode aumentar ou diminuir de acordo com o modelo e dificuldade dos dados - dados + complexos precisam de mais epocas.
model.fit(train_images, train_labels, epochs=5)

Epoch 1/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.6306 - accuracy: 0.7810
Epoch 2/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.3817 - accuracy: 0.8638
Epoch 3/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.3401 - accuracy: 0.8753
Epoch 4/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.3117 - accuracy: 0.8854
Epoch 5/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.2883 - accuracy: 0.8926


Acima cada época foi treinada e apresenta a quantidadde de elementos sendo treinados o valor de erro e a acurácia.  
Na primeira época temos 78% de acurácia e no total esse modelo tem 89,26% de acurácia. Acurácia para nossos dados de treino.  
Observar a evolução no crescimento da acurácia, da primeira para a segunda 8% de aumento da acurácia, da segunda para a terceira foi 1% e assim vai reduzindo até menos de 1%. Sendo assim, mais épocas poderia ocasionar um overfitting. Posso ter uma métrica muito boa para treino mas metricas nao tao boas para teste.

In [14]:
# Avaliando o conjunto de dados sobre os dados de teste
test_loss, test_acc = model.evaluate(test_images, test_labels)

print('Test accuracy:', test_acc)

313/313 [==============================] - 1s 1ms/step - loss: 0.3439 - accuracy: 0.8758
Test accuracy: 0.8758000135421753


Acima o modelo avalia as imagens de teste e seus rótulos e irá guardar tanto o valor de erro na variável test_loss quanto o valor de acurácia na variável test_acc.  
A acurácia do conjunto de teste foi de 87,58%  
Uma acurácia elevada e com isso já posso treinar o modelo.

In [15]:
# Avaliando o modelo de forma individual com o método predict
# Cria uma predição para cada uma das imagens de test e armazena na variável.
predictions = model.predict(test_images)

In [16]:
# imprimindo a primeira predição
predictions[0]

array([1.8607124e-04, 2.0716036e-06, 1.6939612e-05, 2.4970493e-06,
       2.3190431e-05, 3.0099398e-02, 2.2312635e-05, 1.4840637e-02,
       2.4410307e-03, 9.5236588e-01], dtype=float32)

Retorna um array com 10 elementos cada um deles números em notação científica. Todos eles bem próximos a 0, valores bem pequenos (exceto 1).   
A classificação se dará pelo maior valor de predição que teremos dentre esses predictions. Ao inves de ter que olhar e verificar cada um desses valores nós podemos utilizar o NymPy para verificar o índice cujo o elemento é o máximo.

In [17]:
np.argmax(predictions[0])

9

O elemento de índice 9 é o máximo. A classe 9 é a que corresponde a nossa classficiação. Se for comparar o rotulo da imagem 0 teremos que o rótulo dela é 9. Para a primeira imagem de test nosso modelo acertou a predição.  


In [18]:
test_labels[0]

9

In [19]:
# avaliação separada
img = test_images[0]

print(img.shape)

(28, 28)


Acima.  
 Se quiser fazer avaliação separada, individual de cada uma das imagens posso utilizar imagem por imagem.
Coloco a primeira imagem de test em uma variavel separada.  
Printando vejo que a primeira imag de test tem 28 por 28. O modelo espera que um vetor de img seja passado e aqui tenho apenas uma img. Utilizo Numpy para expandir a dimensão dessa imagem e ter um vetor como única imagem. Utilizo o comando abaixo

In [21]:
# img recebe um vetor de imagens
img = (np.expand_dims(img,0))
print(img.shape)

(1, 1, 28, 28)


vetor unitário com 1 img de 28 por 28

In [22]:
# aplico a img com dimensão expandida ao modelo já que se espera um vetor de imgs
# armazeno na predictions_single
predictions_single = model.predict(img)

print(predictions_single)

[[1.8607124e-04 2.0716036e-06 1.6939612e-05 2.4970445e-06 2.3190431e-05
  3.0099425e-02 2.2312572e-05 1.4840637e-02 2.4410319e-03 9.5236588e-01]]


Tenho a predição de uma única imagem  
[[1.8607124e-04 2.0716036e-06 1.6939612e-05 2.4970445e-06 2.3190431e-05
  3.0099425e-02 2.2312572e-05 1.4840637e-02 2.4410319e-03 9.5236588e-01]]

In [23]:
# faço as predições novamente e tenho o mesmo resultado
np.argmax(predictions_single[0])

9